# Model experimentation

In [19]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import STL
from sklearn.base import TransformerMixin, BaseEstimator, RegressorMixin
from feature_engine.creation import CyclicalFeatures
from feature_engine.datetime import DatetimeFeatures
from feature_engine.imputation import DropMissingData
from feature_engine.selection import DropFeatures
from feature_engine.timeseries.forecasting import (
    LagFeatures,
    WindowFeatures,
)
from feature_engine.encoding import OneHotEncoder
from feature_engine.wrappers import SklearnTransformerWrapper

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
#from hybrid_regressor import CombinedRegressor, LinearBoost

import optuna
from optuna.samplers import TPESampler
optuna.logging.set_verbosity(optuna.logging.WARNING)
from mizani.breaks import date_breaks
from plotnine import *
import lightgbm as lgbm

In [2]:
reservoir_hourly = pd.read_csv('../data/processed/reservoir_data_hourly.csv', parse_dates=['timestamp'])
reservoir_hourly = reservoir_hourly.set_index('timestamp')
reservoir_hourly

,nombre_embalse,cota
timestamp,,
2022-01-01 04:00:00+00:00,ANGOSTURA,316.42
2022-01-01 05:00:00+00:00,ANGOSTURA,316.42
2022-01-01 06:00:00+00:00,ANGOSTURA,316.44
2022-01-01 07:00:00+00:00,ANGOSTURA,316.45
2022-01-01 08:00:00+00:00,ANGOSTURA,316.46
...,...,...
2023-01-28 11:00:00+00:00,RAPEL,103.72
2023-01-28 12:00:00+00:00,RAPEL,103.72
2023-01-28 13:00:00+00:00,RAPEL,103.72


## Anomaly detection, removal and imputation

In [3]:
def detect_outliers(df, value_col, period = None, robust = True):
    serie = df[value_col]
    res = STL(serie, period = period, robust = robust).fit()
    resid = res.resid
    q1 = resid.quantile(0.25)
    q3 = resid.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - (3*iqr)
    upper = q3 + (3*iqr)

    anomalies = serie[(resid < lower) | (resid >= upper)]
    df = df.assign(anomaly = np.where(df[value_col].index.isin(anomalies.index), 1, 0))
    df["value_corrected"] = np.where(df["anomaly"] == True, np.NaN, df[value_col])
    df.interpolate(method = "linear", inplace=True)
    df["value_corrected"] = np.where(df["value_corrected"].isna(), df[value_col], df["value_corrected"])
    return df

In [4]:
reservoir_list = reservoir_hourly["nombre_embalse"].unique()

emb_df_list = list()
emb_df_anomalies_list = list()
for emb in reservoir_list:
    emb_df = reservoir_hourly[reservoir_hourly["nombre_embalse"] == emb]
    emb_df = emb_df.asfreq("H")
    emb_df = emb_df.sort_index()
    
    emb_df_sin_outliers = detect_outliers(emb_df, "cota", robust=True)
    emb_df_list.append(emb_df_sin_outliers)

In [5]:
data_cleaned = pd.concat(emb_df_list, axis = 0)
data_cleaned.head()

,nombre_embalse,cota,anomaly,value_corrected
timestamp,,,,
2022-01-01 04:00:00+00:00,ANGOSTURA,316.42,0,316.42
2022-01-01 05:00:00+00:00,ANGOSTURA,316.42,0,316.42
2022-01-01 06:00:00+00:00,ANGOSTURA,316.44,0,316.44
2022-01-01 07:00:00+00:00,ANGOSTURA,316.45,0,316.45
2022-01-01 08:00:00+00:00,ANGOSTURA,316.46,0,316.46


In [6]:
# Validation split
val_len = 24 # one day
in_sample_df = data_cleaned.groupby("nombre_embalse", group_keys=False).apply(lambda x : x.iloc[:-val_len, :])
out_of_sample_df = data_cleaned.groupby("nombre_embalse", group_keys=False).apply(lambda x : x.iloc[-val_len:, :])

in_sample_df = in_sample_df.reset_index()
out_of_sample_df = out_of_sample_df.reset_index()

In [7]:
# Train test split
test_time = pd.Timedelta(24*7*4, unit = "H")
split_point = in_sample_df["timestamp"].max() - test_time

X_train = in_sample_df[in_sample_df["timestamp"] < split_point]
X_test = in_sample_df[in_sample_df["timestamp"] >= split_point - pd.Timedelta(24*4, unit = "H")]

y_train = in_sample_df[in_sample_df["timestamp"] < split_point][["timestamp","nombre_embalse","value_corrected"]]
y_test = in_sample_df[in_sample_df["timestamp"] >= split_point - pd.Timedelta(24*4, unit = "H")][["timestamp", "nombre_embalse", "value_corrected"]]

In [8]:
X_train = X_train.set_index(["timestamp", "nombre_embalse"])
X_test = X_test.set_index(["timestamp", "nombre_embalse"])
y_train = y_train.set_index(["timestamp", "nombre_embalse"])
y_test = y_test.set_index(["timestamp", "nombre_embalse"])

## Feature Engineering

In [9]:
# Fourier Features Class
class AddFourierFeatures(BaseEstimator, TransformerMixin):
    seconds_per_day = 24*60*60     # Daily dataset
    seconds_per_hour = 60*60       # Hourly dataset

    def __init__(self, K, periods: list, by = "day"):
        self.K = K
        self.periods = periods
        self.by = by

    def fit(self, X, y=None):

        return self

    def transform(self, X, y=None):
        X = X.copy()
        dates = X.index
        
        for period in self.periods:
            term = self.K / period
            timestamps = dates.map(datetime.datetime.timestamp)
            ts_scaled = []

            for ts in timestamps:
                if self.by == "day":
                    x_scaled = round(ts / self.seconds_per_day)
                    ts_scaled.append(x_scaled)
                else:
                    x_scaled = round(ts / self.seconds_per_hour)
                    ts_scaled.append(x_scaled)

            X["fourier_sin"] = [np.sin(2 * np.pi * term * ts) for ts in ts_scaled]
            X["fourier_cos"] = [np.cos(2 * np.pi * term * ts) for ts in ts_scaled]

        return X

In [10]:
# Transformers
horizon = 24

dtf = DatetimeFeatures(
    variables="index",
    features_to_extract=[
        "hour",
        "day_of_month",
        "month",
        "year",
        "day_of_year",
        "week",
        "day_of_week",
        "weekend"
    ],
    drop_original = False,
    utc = True
)

cyclicf = CyclicalFeatures(
    variables=["hour", "month", "day_of_year"],
    drop_original= True
)

fourierf = AddFourierFeatures(
    K = 1,
    periods=[horizon, horizon*2],
    by = "hour"
)

lagf = LagFeatures(
    variables="value_corrected",
    periods=list(range(1,horizon+1)),
    missing_values = "ignore"
)

windf24 = WindowFeatures(
    variables="value_corrected",
    functions=["mean"],
    window=[int(horizon/2), horizon],
    freq="1H",
    missing_values="ignore"
)

imputer = DropMissingData()

drop_features = DropFeatures(features_to_drop=["value_corrected"])

In [11]:
prep_pipeline = Pipeline([
    ("datetime features", dtf),
    ("cyclical features", cyclicf),
    ("fourier features", fourierf),
    ("lag features", lagf),
    ("window features 24H", windf24),
    ("imputer", imputer),
    ("drop features", drop_features)
])

centrales = X_train.index.get_level_values(1).unique()
X_train_t_list = list()
X_test_t_list = list()
for cen in centrales:
    emb_df = X_train[X_train.index.get_level_values(1) == cen]
    emb_df = emb_df.reset_index(level=1)
    emb_df_t = prep_pipeline.fit_transform(emb_df)
    emb_df_t = emb_df_t.set_index("nombre_embalse", append=True)
    X_train_t_list.append(emb_df_t)
    
    emb_test = X_test[X_test.index.get_level_values(1) == cen]
    emb_test = emb_test.reset_index(level=1)
    emb_test_t = prep_pipeline.transform(emb_test)
    emb_test_t = emb_test_t.set_index("nombre_embalse", append=True)
    X_test_t_list.append(emb_test_t)

X_train_t = pd.concat(X_train_t_list, axis = 0)
X_test_t = pd.concat(X_test_t_list, axis = 0)

In [12]:
# Align
y_train_t = y_train.loc[X_train_t.index]
y_test_t = y_test.loc[X_test_t.index]

In [13]:
X_train_t = X_train_t.reset_index(level=1)
X_train_t["nombre_embalse"] = X_train_t["nombre_embalse"].astype("category")
X_test_t = X_test_t.reset_index(level=1)
X_test_t["nombre_embalse"] = X_test_t["nombre_embalse"].astype("category")

In [14]:
numeric_features = [var for var in X_train_t.columns if ((X_train_t[var].dtype == 'float') | (X_train_t[var].dtype == 'int')) & (var != "value_corrected")]
categorical_features = [var for var in X_train_t.columns if ((X_train_t[var].dtype == 'O') | (X_train_t[var].dtype == 'category'))]

In [15]:
categorical_features

['nombre_embalse']

In [16]:
# Transform features
scaler = SklearnTransformerWrapper(transformer=StandardScaler(),
                                   variables=numeric_features)

encoder = OneHotEncoder(variables=categorical_features)

transformer_pipe = Pipeline([
    ("scaler", scaler),
    ("encoder", encoder)
])

X_train_prep = transformer_pipe.fit_transform(X_train_t)
X_test_prep = transformer_pipe.transform(X_test_t)

In [17]:
X_train_prep.shape, y_train_t.shape

((104388, 53), (104388, 1))

In [20]:
# Building Custom Model
## Credits -> 
class CombinedRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, 
                 base_regressor=lgbm.LGBMRegressor, 
                 backup_regressor=LinearRegression, 
                 lower=0.1, 
                 upper=1.9,
                 random_state=None,
                 **kwargs):
        self.base_regressor = base_regressor()
        self.backup_regressor = backup_regressor()
        
        self.set_random_state(random_state)
        
        self.lower = lower
        self.upper = upper
        
        self.set_params(**kwargs)
        
    def fit(self, X, y):
        self.base_regressor.fit(X, y)
        self.backup_regressor.fit(X, y)
        return self
    
    def predict(self, X, y=None):
        y_base = self.base_regressor.predict(X)
        y_base = y_base.reshape(-1,1)
        y_backup = self.backup_regressor.predict(X)
        y_pred = np.where((self.lower * y_backup <= y_base) & (y_base <= self.upper * y_backup), 
                          y_base,
                          y_backup)
        
        return y_pred
    
    def __repr__(self):
        # not as good as sklearn pretty printing,
        # but shows updated params of subestimator
        return f'CombinedRegressor({self.get_params()})'
    
    def get_params(self, deep=False, **kwargs):
        base_regressor_params = self.base_regressor.get_params(**kwargs)
        # remove random state as it should be a global param of the estimator
        base_regressor_params.pop('random_state', None)
        base_regressor_params = {'base_regressor__' + key: value 
                                 for key, value 
                                 in base_regressor_params.items()}
        
        backup_regressor_params = self.backup_regressor.get_params(**kwargs)
        backup_regressor_params.pop('random_state', None)
        backup_regressor_params = {'backup_regressor__' + key: value 
                                   for key, value 
                                   in backup_regressor_params.items()}
        
        own_params = {
            'lower': self.lower,
            'upper': self.upper,
            'random_state': self.random_state
        }
        
        params = {**own_params,
                  **base_regressor_params, 
                  **backup_regressor_params, 
                 }
        
        if deep:
            params['base_regressor'] = self.base_regressor
            params['backup_regressor'] = self.backup_regressor
        return params
    
    def set_random_state(self, value):
        self.random_state = value
        if 'random_state' in self.base_regressor.get_params().keys():
            self.base_regressor.set_params(random_state=value)
        # linear reg does not have random state, but just in case..
        if 'random_state' in self.backup_regressor.get_params().keys():
            self.backup_regressor.set_params(random_state=value)
    
    def set_params(self, **params):
        for key, value in params.items():
            if key.startswith('base_regressor__'):
                trunc_key = {key[len('base_regressor__'):]: value}
                self.base_regressor.set_params(**trunc_key)
            elif key.startswith('backup_regressor__'):
                trunc_key = {key[len('backup_regressor__'):]: value}
                self.backup_regressor.set_params(**trunc_key)
            elif key == 'random_state':
                self.set_random_state(value)
            else:
                # try to fetch old value first to raise AttributeError
                # if not exists
                old_value = getattr(self, key)
                setattr(self, key, value)
        # set_params needs to return self to make gridsearch work
        return self
        
    def _more_tags(self):
        # no_validation added because validation is happening 
        # within built-in sklearn estimators
        return {**self.base_regressor._more_tags(), 'no_validation': True}

NameError: name 'LinearRegression' is not defined